In [1]:
import cv2
import numpy as np

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
# config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.8
session = InteractiveSession(config=config)

from gen_net import GEN

Using TensorFlow backend.


In [2]:
fname = 'UBMk30rjy0o.mp4'
path = 'data/val/' + fname

In [3]:
cap = cv2.VideoCapture(path)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 動画の画面横幅
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 動画の画面縦幅
fcnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # 総フレーム数
fps = cap.get(cv2.CAP_PROP_FPS)
print(w, h, fcnt, fps)

1920 1080 30500 25.0


In [9]:
for _ in range(1200):
    ret, frame = cap.read()
    
r = 2
low_img = cv2.resize(frame , (int(w/r), int(h/r)))

In [10]:
# input_shape = (h, w, 3)
input_shape = low_img.shape
model_path = 'models/20200515_generator.h5'

gen = GEN(model_path, input_shape)

model = gen.generator

In [11]:
pix = 0
for l in model.layers:
    s = l.output_shape
    pix += s[1]*s[2]*s[3]

In [12]:
pix * 32 / 1024 / 1024 / 1024 / 8

3.057074546813965

In [13]:
img = low_img

In [14]:
pred = model.predict(((img - 127.5) / 127.5)[np.newaxis, :, :, :])[0]
# out =  ((pred - pred.min()) / (pred.max() - pred.min()) * 255)
out = pred * 127.5 + 127.5

In [15]:
cv2.imwrite('test.jpg', out)

True

In [16]:
img_hsv = cv2.cvtColor(out,cv2.COLOR_BGR2HSV)  # 色空間をBGRからHSVに変換
s_magnification = 1.2# 彩度(Saturation)の倍率
v_magnification = 0.9  # 明度(Value)の倍率

img_hsv[:,:,(1)] = img_hsv[:,:,(1)]*s_magnification  # 彩度の計算
img_hsv[:,:,(2)] = img_hsv[:,:,(2)]*v_magnification  # 明度の計算
img_bgr = cv2.cvtColor(img_hsv,cv2.COLOR_HSV2BGR)  # 色空間をHSVからBGRに変換

cv2.imwrite('rose_contrast.jpg',img_bgr) 

True